# PRMT-2517 Emis spine errors from MI data - Dec 2021

In [2]:
import pandas as pd
import numpy as np
import paths, data
from data.practice_metadata import read_asid_metadata

In [3]:
def convert_to_int(val):
    try:
        return int(val)
    except:
        return val

mi_data_file_location = "s3://prm-gp2gp-notebook-data-prod/PRMT-2517-EMIS-spine-errors-MI-Dec/MI_SR-Dec_2021.csv"

dates_fields = ["RequestTime", "RequestAckTime", "ExtractFailureTime", "ExtractTime", "ConversationCloseTime"]
practice_registrations = pd.read_csv(mi_data_file_location, parse_dates=dates_fields).fillna("None")

practice_registrations["ExtractFailurePoint"] = practice_registrations["ExtractFailurePoint"].apply(convert_to_int)
practice_registrations["ExtractFailureType"] = practice_registrations["ExtractFailureType"].apply(convert_to_int)
practice_registrations["ExtractFailureErrorCode"] = practice_registrations["ExtractFailureErrorCode"].apply(convert_to_int)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
practice_registrations = (
    practice_registrations
        .sort_values(by="_time", ascending=True)
        .drop_duplicates(subset=["ConversationID"], keep="last")
    )

practice_registrations["ConversationIdLower"] = practice_registrations["ConversationID"].str.lower()

In [5]:
emis_errors_file_location = "s3://prm-gp2gp-notebook-data-prod/PRMT-2517-EMIS-spine-errors-MI-Dec/EMIS-core-extract-not-sent-spine-errors-Dec-2021.csv"

emis_errors = pd.read_csv(emis_errors_file_location, parse_dates=["Date Requested"]).fillna("None")

emis_errors["ConversationIdLower"] = emis_errors["Conversation ID"].str.lower()


In [6]:
enhanced_left = emis_errors.merge(
        practice_registrations[
            ['ConversationIdLower',
             'ExtractFailurePoint', 
             'ExtractFailureType',
             'ExtractFailureErrorCode',
             'ExtractFailureErrorDescription',
             'RequestAckCode',
             'RequestAckErrorCode',
             'RequestAckErrorDescription',
             'RequestAckFailureType',
             'RequestorApplicationStatus'
            ]], 
    how = "left", 
    on="ConversationIdLower")


In [7]:
enhanced_left[['ExtractFailurePoint','ExtractFailureType','ExtractFailureErrorCode']].value_counts()

ExtractFailurePoint  ExtractFailureType  ExtractFailureErrorCode
60.0                 4                    20                        1090
20.0                 3                   -31                           3
70.0                 4                    20                           2
dtype: int64

In [8]:
with pd.ExcelWriter("PRMT-2517-Emis-Spine-Errors-MI-Dec-2021.xlsx") as writer:
     enhanced_left.to_excel(writer, sheet_name="Spine errors Dec 2021",index=False)